<a href="https://colab.research.google.com/github/GuillBla/Internship_Report_Code/blob/main/Mixed_Inputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A script to execute to prevent from the limits of imposed by Google Colab

In [ ]:
# function ConnectButton(){
#     console.log("Connect pushed"); 
#     document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
# }
# setInterval(ConnectButton,60000);

In [ ]:
!pip install split-folders
!pip install livelossplot

In [ ]:
import splitfolders
from google.colab import drive
import zipfile
import os,shutil
from os import listdir
from os.path import isfile, join
import csv
import os
import requests
from bs4 import BeautifulSoup
import re
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop, Adam
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten,Conv2D, GlobalAveragePooling2D
from keras import regularizers
from pathlib import Path
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import concatenate
import numpy as np
import argparse
import locale
import os
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

In [ ]:
drive.mount('/content/drive')

If the following folders have already been created, they need to be removed (uncomment if it is the case)

In [ ]:
# # Clear the environment
# shutil.rmtree(r'/content/topic_classification_splitted')
# shutil.rmtree(r'/content/topic_classification')

In [ ]:
zip_ref = zipfile.ZipFile('/content/drive/MyDrive/topic_classification.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/content') #Extracts the files into the /tmp folder
zip_ref.close()

I split the dataset into two folders : 
1. One containing the training set : /content/topic_classification_splitted/train
2. One containing the testing set : /content/topic_classification_splitted/test

Note that I will split the first folder in two to create a validation set.

In [ ]:
splitfolders.fixed(r"/content/topic_classification", output=r"/content/topic_classification_splitted",
    seed=2504, fixed=(1, 0, 0), oversample=False, group_prefix=None, move=False)

# os.rename("/content/topic_classification_splitted/val", "/content/topic_classification_splitted/test")

names_classes = [f for f in listdir('/content/topic_classification_splitted/test')]

Extracting HMTL Input 

In [ ]:
# The topics of the dataset we are looking for :
topics = ["Accommodation", "Administration", "Audiovisual", "Automotive", "Casino", "Dating", "Drugs", "Engineering",
          "Fashion", "Finance", "Food", "Health", "Office", "Software", "Insurance", "Laboratory", "Lawyer",
          "Entertainment", "Logistic", "Marketing", "Personal","Skills", "Politic", "Porn", "Religion", "Sustainability",
          "Technology"]

dictionnary = {"Accommodation" :1 , "Administration":2, "Audiovisual":3, "Automotive":4, "Casino & Gambling":5, "Dating":6, "Drugs":7, "Engineering & Building":8,
          "Fashion":9, "Finance":10, "Food & Drinks":11, "Health & Wellness":12, "Home & Office":13, "Hosting & Software":14, "Insurance":15, "Laboratory":16, "Lawyer":17,
          "Leisure & Entertainment":18, "Logistic":19, "Marketing":20, "Personal":21, "Personal Skills":22, "Politic":23, "Porn":24, "Religion":25, "Sustainability":26,
          "Technology & Electronic":27}

path = "/content/topic_classification_splitted"

blacklist = ['[document]', 'noscript', 'html', 'meta', 'input', 'script']

# A function to search for a pattern : key, in a string : content
def recherche(key, content, extract=False):
    nkey = 0
    content = content.lower()
    key = key.lower()
    for k in range(len(content)):
        if content[k:k + len(key)] == key:
            nkey += 1
    return nkey

with open('html_input.csv', 'w', newline='') as file: 
  writer = csv.writer(file)

  for (root, dirs, file) in os.walk(path):
    for url in file:
      if '.jpg' in url:
        # Cleaning the names of the files to urls
        url = url.replace('http_', "http://")
        url = url.replace('https_', "https://")
        url = url.replace('https_www', "https://")
        url = url.replace('http_www', "http://")
        url = url.replace('.jpg', "")

        # HTML request :
        try:
          res = requests.get(url)
          html_page = res.content
          soup = BeautifulSoup(html_page, 'html.parser')
          puretext = soup.find_all(text=True)
          htmltext = soup.find_all()
          htmloutput = ''

          for t in htmltext:
            if t.parent.name not in blacklist:
              htmloutput += '{} '.format(t)
       
          #retrieve true topic : 
          category = re.split("/", root, 4)[-1] 
          row=[dictionnary[category],url]
          # Looking for possible topics in HTML code and adding the data to the csv file
          for topic in topics:
            row.append(recherche(topic, htmloutput))
          writer.writerow(row)
        except Exception as e:
          break

Read the .csv file with the HMTL extraction results as a DataFrame

In [ ]:
def load_html_attributes(inputPath):
	# initialize the list of column names in the CSV file and then
	# load it using Pandas
	cols = ["category","url", "Accommodation", "Administration", "Audiovisual", "Automotive", "Casino & Gambling", "Dating", "Drugs", "Engineering & Building",
          "Fashion", "Finance", "Food & Drinks", "Health & Wellness", "Home & Office", "Hosting & Software", "Insurance", "Laboratory", "Lawyer",
          "Leisure & Entertainment", "Logistic", "Marketing", "Personal", "Personal Skills", "Politic", "Porn", "Religion", "Sustainability",
          "Technology & Electronic"]
	df = pd.read_csv(inputPath, sep=",", header=None, names=cols)
	
	return df

Creating data from images

In [ ]:
def load_images(df, inputPath):
	images = []
	data_path = inputPath

	for category in listdir(data_path):
		for image in listdir(data_path+ "/" + category):
			url = image
			url = url.replace('http_', "http://")
			url = url.replace('https_', "https://")
			url = url.replace('https_www', "https://")
			url = url.replace('http_www', "http://")
			url = url.replace('.jpg', "")
			if (any(df['url'] == url)):
				img = cv2.imread(data_path + "/" + category + "/" + image , cv2.IMREAD_COLOR)
				# resize makes problems but this solves it :
				try:
					img_resize = cv2.resize(img,(224,224))
					images.append(img_resize)
				except:
					continue

	img_data = np.array(images)
	img_data = img_data.astype('float32')
	img_data /= 255
	return img_data


Functions two create the two models that we will link

In [ ]:
# Create Multi-Layer Perceptron
def create_mlp(dim):
 model = Sequential()
 model.add(Dense(8, input_dim=dim, activation="relu"))
 model.add(Dense(4, activation="relu"))
 return model

def create_cnn(input_shape, n_classes):
  base_model = Xception(weights='imagenet', include_top=False, input_shape=input_shape)
  model = base_model.output
  model = GlobalAveragePooling2D()(model)
  model = Dense(256, activation="relu", name="HiddenLayer1")(model)
  model = Dropout(0.5)(model)
  model = Dense(128, activation="relu", name="HiddenLayer2", kernel_regularizer=regularizers.l2(0.01))(model)
  model = Dropout(0.2)(model)
  model = Dense(27, activation="softmax", name="OutputLayer")(model)

  model = Model(inputs=base_model.input, outputs=model)
  return model

In [ ]:
# df is a Dataframe containing all of the HTML informations
print("[INFO] loading html attributes...")
inputPath = r"/content/html_input.csv"
# df = datasets.load_html_attributes(inputPath)
df_preprocess = load_html_attributes(inputPath)
df_training = df_preprocess
#delete "url" column
df_training = df_training.drop(['url'], axis=1).astype(np.float32)

# images is a list of the images of the dataset
print("[INFO] loading images...")
# images = datasets.load_house_images(df, r"/content/topic_classification_splitted/test")
images = load_images(df_preprocess, r"/content/topic_classification_splitted/train")

Split the dataset into train and test splits

In [ ]:
# partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
print("[INFO] processing data...")
split = train_test_split(df_training, images, test_size=0.2, random_state=2504)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split

trainY = trainAttrX["category"]
testY = testAttrX["category"] 

In [ ]:
# create the MLP and CNN models
mlp = create_mlp(trainAttrX.shape[1])

input_shape = (224, 224, 3)
n_classes=27
cnn = create_cnn(input_shape, n_classes)

# create the input to our final set of layers as the *output* of both
# the MLP and CNN
combinedInput = concatenate([mlp.output, cnn.output])

x = Dense(4, activation="relu")(combinedInput)
x = Dense(27, activation="softmax")(x)

model = Model(inputs=[mlp.input, cnn.input], outputs=x)

Plot the model

In [ ]:
# Imports
import matplotlib.pyplot as plt
from tensorflow.keras.utils import plot_model

# Plot the model
plot_model(model, to_file='model.png')

# Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()

In [ ]:
opt = Adam(learning_rate=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
model.fit(
	x=[trainAttrX, trainImagesX], y=trainY,
	validation_data=([testAttrX, testImagesX], testY),
	epochs=30, batch_size=8)


In [ ]:
# Predict on test set
print("[INFO] Computing metrics...")
preds = model.predict([testAttrX, testImagesX])
prediction = []
for pred in preds:
  max = 0
  best = -1
  for p in range(len(pred)):
    if pred[p]> max:
      max = pred[p]
      best = p
  prediction.append(best)

print("Predicted classes: ")
print(prediction)
true_classes = []
for i in range(len(prediction)):
  true_classes.append(testY.iloc[i])
print("True classes: ")
print(true_classes)

diff = np.array(prediction) - np.array(true_classes)
sum=0
for i in range (len(diff)):
  if diff[i] == 0:
    sum +=1
accuracy = sum/len(diff)
print("Accuracy : "+ str(accuracy))